# MILESTONE 6. LAGRANGE POINTS AND THEIR STABILITY

## a) Write a high order embedded Runge-Kutta method

Se va a obtener el método Runge-Kutta-Fehlberg RKF45.Empleando la tabla de Butcher para sacar los coeficientes:

In [1]:
import numpy as np

def rkf45_step(F, U, t, dt):


    k1 = F(U, t)
    k2 = F(U + dt*(1/4)*k1, t + dt*(1/4))

    k3 = F(U + dt*(3/32*k1 + 9/32*k2),
           t + 3*dt/8)

    k4 = F(U + dt*(1932/2197*k1 - 7200/2197*k2 + 7296/2197*k3),
           t + 12*dt/13)

    k5 = F(U + dt*(439/216*k1 - 8*k2 + 3680/513*k3 - 845/4104*k4),
           t + dt)

    k6 = F(U + dt*(-8/27*k1 + 2*k2 - 3544/2565*k3 +
                   1859/4104*k4 - 11/40*k5),
           t + dt/2)

   
    U_orden5 = U + dt*(16/135*k1 + 6656/12825*k3 +
                     28561/56430*k4 - 9/50*k5 + 2/55*k6)

    # ---- Solución embebida orden 4
    U_orden4 = U + dt*(25/216*k1 + 1408/2565*k3 +
                    2197/4104*k4 - 1/5*k5)

    # ---- Error estimado
    err = np.linalg.norm(U_orden5 - U_orden4)

    return U_orden5, U_orden4, err


## b) Write a function to simulate the circular restricted three body problem

La masa del tercer cuerpo (m_3) se desprecia

In [2]:
import numpy as np

def function_cir_res_3bp (U, t, mu):


    x, y, v_x, v_y = U

    r_1 = np.sqrt((x+mu)**2+y**2) #Distancia al cuerpo m_1 
    r_2 = np.sqrt((x-1+mu)**2+y**2) #Distancia al cuerpo m_2

    dU_dx =  x - (1-mu)*(x+mu)/r_1**3 - mu*(x-1+mu)/r_2**3
    dU_dy = y - (1-mu)*y/r_1**3 - mu*y/r_2**3

    dx_dt = v_x
    dy_dt = v_y
    dv_x_dt = 2*v_y + dU_dx
    dv_y_dt = -2*v_x + dU_dy

    return np.array([dx_dt, dy_dt, dv_x_dt, dv_y_dt])


### c) Determination of the Lagrange Points

En los puntos de equilibrio: $\dot{x} = \dot{y} = \dot{v_x} = \dot{v_y} = 0$. Sustituyendo en las ecuaciones del problema restringido circular de tres cuerpos:


$f(x,y) = \left( \frac{\partial \Omega}{\partial x}(x, y), \frac{\partial \Omega }{\partial y}(x, y) \right) = \vec{0}$


In [5]:
import numpy as np

def gradiente_Omega(x, y, mu):
    r_1 = np.sqrt((x + mu)**2 + y**2)
    r_2 = np.sqrt((x - 1 + mu)**2 + y**2)

    dOmega_dx = x - (1-mu)*(x + mu)/r_1**3 - mu*(x - 1 + mu)/r_2**3
    dOmega_dy = y - (1-mu)*y/r_1**3       - mu*y/r_2**3

    return np.array([dOmega_dx, dOmega_dy])


def f_Lagrange(X, mu):
 
    x, y = X
    return gradiente_Omega(x, y, mu)

# Para sacar los puntos --> f_Lagrange (X) = 0

def newton_2D(f, X0, mu, tol=1e-12, maxiter=30):
    X = np.array(X0, dtype=float)

    for _ in range(maxiter):
        F = f(X, mu)

        if np.linalg.norm(F) < tol:
            return X

        # Jacobiano,( lo tengo en otro modulo)
        h = 1e-6
        J = np.zeros((2,2))
        for i in range(2):
            dX = np.zeros(2)
            dX[i] = h
            J[:, i] = (f(X + dX, mu) - f(X - dX, mu)) / (2*h)

        delta = np.linalg.solve(J, -F)
        X = X + delta

    raise ValueError("Newton no converge")


def dOmega_dx_on_axis(x, mu):
    return gradiente_Omega(x, 0.0, mu)[0]


def newton_1D(g, x0, mu, tol=1e-12, maxiter=30):
    x = x0

    for _ in range(maxiter):
        fx = g(x, mu)

        if abs(fx) < tol:
            return x

        h = 1e-6
        dfx = (g(x+h, mu) - g(x-h, mu)) / (2*h)

        x = x - fx/dfx

    raise ValueError("Newton 1D no converge")

def aproximaciones_iniciales(mu):
    L1_0 = 1 - (mu/3)**(1/3)
    L2_0 = 1 + (mu/3)**(1/3)
    L3_0 = -1 - 5*mu/12
    L4_0 = np.array([0.5 - mu,  np.sqrt(3)/2])
    L5_0 = np.array([0.5 - mu, -np.sqrt(3)/2])
    return L1_0, L2_0, L3_0, L4_0, L5_0


def puntos_Lagrange(mu):
    L1_0, L2_0, L3_0, L4_0, L5_0 = aproximaciones_iniciales(mu)

   
    L1_x = newton_1D(dOmega_dx_on_axis, L1_0, mu)
    L2_x = newton_1D(dOmega_dx_on_axis, L2_0, mu)
    L3_x = newton_1D(dOmega_dx_on_axis, L3_0, mu)

    L1 = np.array([L1_x, 0.0])
    L2 = np.array([L2_x, 0.0])
    L3 = np.array([L3_x, 0.0])

    
    L4 = newton_2D(f_Lagrange, L4_0, mu)
    L5 = newton_2D(f_Lagrange, L5_0, mu)

    return L1, L2, L3, L4, L5


### c) Estabilidad de los puntos de Lagrange 

In [7]:
import numpy as np
import matplotlib as plt 


def F_CR3BP(U, t, mu):
    x, y, vx, vy = U

    r1 = np.sqrt((x + mu)**2 + y**2)
    r2 = np.sqrt((x - 1 + mu)**2 + y**2)

    dOdx = x - (1-mu)*(x + mu)/r1**3 - mu*(x - 1 + mu)/r2**3
    dOdy = y - (1-mu)*y/r1**3       - mu*y/r2**3

    dxdt  = vx
    dydt  = vy
    dvxdt = 2*vy + dOdx
    dvydt = -2*vx + dOdy

    return np.array([dxdt, dydt, dvxdt, dvydt])


def jacobiano_CR3BP(Ueq, mu, h=1e-6):
    n = len(Ueq)
    J = np.zeros((n,n))

    for j in range(n):
        dU = np.zeros(n)
        dU[j] = h
        Fp = F_CR3BP(Ueq + dU, 0, mu)
        Fm = F_CR3BP(Ueq - dU, 0, mu)
        J[:, j] = (Fp - Fm) / (2*h)

    return J


def estabilidad_Lagrange(Xeq, mu):
    Ueq = np.array([Xeq[0], Xeq[1], 0., 0.])
    J = jacobiano_CR3BP(Ueq, mu)
    vals = np.linalg.eigvals(J)

    re = np.real(vals)
    if np.any(re > 0):
        tipo = "Inestable"
    elif np.all(re < 0):
        tipo = "Estable"
    else:
        tipo = "Centro / Neutro"

    return vals, tipo


# =============================================
# (6) VISUALIZACIÓN DE LOS PUNTOS DE LAGRANGE
# =============================================

def plot_Lagrange(mu):
    L1, L2, L3, L4, L5 = puntos_Lagrange(mu)

    prim1 = np.array([-mu, 0])
    prim2 = np.array([1-mu, 0])

    plt.figure(figsize=(7,6))
    plt.scatter(prim1[0], prim1[1], s=200, c='orange', label="Primary 1")
    plt.scatter(prim2[0], prim2[1], s=150, c='brown', label="Primary 2")

    for P, name in zip([L1,L2,L3,L4,L5], ["L1","L2","L3","L4","L5"]):
        plt.scatter(P[0], P[1], s=120, label=name)
        plt.text(P[0]+0.02, P[1]+0.02, name)

    plt.title("Puntos de Lagrange en el CR3BP")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.legend()
    plt.axis("equal")
    plt.show()

    return (L1, L2, L3, L4, L5)


# =============================================
# (7) VISUALIZACIÓN DE ESTABILIDAD (AUTOVALORES)
# =============================================

def plot_estabilidad(mu):
    Ls = puntos_Lagrange(mu)
    nombres = ["L1","L2","L3","L4","L5"]

    plt.figure(figsize=(8,6))

    for P, name in zip(Ls, nombres):
        vals, tipo = estabilidad_Lagrange(P, mu)
        plt.scatter(np.real(vals), np.imag(vals), label=f"{name} ({tipo})")

    plt.axvline(0, color='k', linewidth=1)
    plt.title("Autovalores del Jacobiano en cada punto de Lagrange")
    plt.xlabel("Re(λ)")
    plt.ylabel("Im(λ)")
    plt.grid(True)
    plt.legend()
    plt.show()

### d) Órbitas alrededor de puntos de Lagrange con diferentes esquemas temporales

In [8]:
import numpy as np
from temporal_schemes_v2 import (step_euler, step_inverse_euler, step_crank_nicholson, step_rk4, Leap_Frog)
import matplotlib.pyplot as plt


def integrate_scheme(F, U0, mu, T, N, step_method):
    t = np.linspace(0, T, N)
    dt = t[1] - t[0]

    U = np.zeros((N, len(U0)))
    U[0] = U0

    for n in range(N-1):
        U[n+1] = step_method(F, U[n], t[n], dt, mu)

    return U[:,0], U[:,1]   # coordenadas x(t), y(t)

def orbit_around_point(F, point, mu, perturb, T, N, step_method):
    U0 = np.array([point[0], point[1], 0, 0]) + np.array(perturb)
    return integrate_scheme(F, U0, mu, T, N, step_method)

def plot_orbits_by_schemes(F, point, mu, name, perturb=[1e-3,0,0,0], T=20, N=20000):

    schemes = {
        "Euler": step_euler,
        "Euler Inverso": step_inverse_euler,
        "Crank–Nicolson": step_crank_nicolson,
        "RK4": step_rk4,
    }

    plt.figure(figsize=(7,6))

    for sname, method in schemes.items():
        x, y = orbit_around_point(F, point, mu, perturb, T, N, method)
        plt.plot(x, y, label=sname)

    plt.scatter([point[0]], [point[1]], c='red', s=60, label=f"{name}")

    plt.title(f"Órbitas alrededor de {name} mediante distintos esquemas")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.axis("equal")
    plt.grid(True)
    plt.legend()
    plt.show()
